# Set-up

In [2]:
import os
import sys
import glob
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from PyComplexHeatmap import *

In [3]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [11]:
results_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results"
dataset_name = "meta_analysis"
in_date = "03Sep23"

# Load in topics

In [9]:
topic_2 = pd.read_csv("/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/meta_analysis/03Sep23/topics/Topic_2/topic_gene_weights.tsv", sep="\t", index_col=0)
topic_3 = pd.read_csv("/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/meta_analysis/03Sep23/topics/Topic_3/topic_gene_weights.tsv", sep="\t", index_col=0)
topic_4 = pd.read_csv("/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/meta_analysis/03Sep23/topics/Topic_4/topic_gene_weights.tsv", sep="\t", index_col=0)

In [26]:
topic_3.iloc[:25]

,rank_weight,rank
Zfhx3,2396.526696,1.0
Mef2a,2271.030849,2.0
Chd9,2095.107969,3.0
Maml3,1929.708330,4.0
Mir99ahg,1922.264236,5.0
Mef2c,1773.106968,6.0
Zeb2,1757.690498,7.0
Foxn3,1573.355997,8.0
Ikzf1,1454.137414,9.0
Jmjd1c,1395.627152,10.0


# GRN metadata

In [ ]:
# Read in all the network metadata
grn_metadata_df = pd.read_csv(os.path.join(results_dir, dataset_name, in_date, "cleaned_grn_metadata.tsv"), sep="\t", index_col=0)
len(grn_metadata_df)

120

In [ ]:
grn_metadata_df = grn_metadata_df[(grn_metadata_df["genotype"] != "both") & (grn_metadata_df["method"] == "celloracle") & (grn_metadata_df["normalization"] == "pf_log1p_pf") & (grn_metadata_df["dataset"] == "igvf_b01_LeftCortex")]
len(grn_metadata_df)

2

# Load in GRN network scores

In [13]:
# Read in the network scores
tf_scores_df = pd.read_csv(os.path.join(results_dir, dataset_name, in_date, "tf_only_network_scores.tsv"), sep="\t").rename(columns={"Unnamed: 0": "tf_name"})
tf_scores_df.head()

,tf_name,degree_all,degree_centrality_all,degree_in,degree_centrality_in,degree_out,degree_centrality_out,betweenness_centrality,eigenvector_centrality,grn_name
0,Runx1t1,2,0.004866,0,0.000000,2,0.004866,0.0,0.005795,Bridge_Satpathy_aracne_balanced_genotype_micro...
1,Runx1t1,50,0.142045,2,0.005682,48,0.136364,81.0,0.226653,Bridge_Satpathy_aracne_balanced_genotype_micro...
2,Runx1t1,37,0.099196,1,0.002681,36,0.096515,24.0,0.183886,Bridge_Satpathy_aracne_balanced_genotype_micro...
3,Runx1t1,1,0.001031,1,0.001031,0,0.000000,0.0,0.014566,Bridge_Satpathy_celloracle_balanced_genotype_m...
4,Runx1t1,1,0.001211,1,0.001211,0,0.000000,0.0,0.015317,Bridge_Satpathy_celloracle_balanced_genotype_m...


In [17]:
# Keep only the networks that we want to compare
tf_scores_df = tf_scores_df[tf_scores_df["grn_name"].isin(grn_metadata_df.index)]

In [19]:
tf_scores_df.sort_values("degree_centrality_all", ascending=False).head()

,tf_name,degree_all,degree_centrality_all,degree_in,degree_centrality_in,degree_out,degree_centrality_out,betweenness_centrality,eigenvector_centrality,grn_name
11244,Mef2a,66,0.091922,8,0.011142,58,0.080780,4699.0,0.910137,igvf_b01_LeftCortex_celloracle_balanced_genoty...
15987,Klf12,62,0.086351,0,0.000000,62,0.086351,0.0,0.548276,igvf_b01_LeftCortex_celloracle_balanced_genoty...
11243,Mef2a,57,0.076203,2,0.002674,55,0.073529,228.0,1.000000,igvf_b01_LeftCortex_celloracle_balanced_genoty...
13795,Mef2c,48,0.064171,11,0.014706,37,0.049465,1591.0,0.783146,igvf_b01_LeftCortex_celloracle_balanced_genoty...
21771,Foxn3,46,0.064067,8,0.011142,38,0.052925,622.0,0.469679,igvf_b01_LeftCortex_celloracle_balanced_genoty...
